# 📚 Building Your First AI Chatbot - Interactive Tutorial

Welcome! This notebook will guide you through building an AI chatbot that can answer questions about PDF documents.

## 🎯 What You'll Learn
1. How LLMs (Large Language Models) work
2. Processing PDF documents
3. Context Augmented Generation (CAG)
4. Interacting with AI (locally with Ollama or via OpenAI API)
5. Building a complete chatbot

## 📋 Prerequisites
- Basic Python knowledge
- **Ollama installed** (free local models - recommended!) OR OpenAI API key
- A PDF file to test with

## 🆓 This Tutorial Uses Ollama (Free!)
We'll use Ollama to run AI models locally on your computer - no API costs, no usage limits!
Already installed Ollama and pulled llama3.2? Great! Otherwise, see `../OLLAMA_SETUP.md`.

## Step 1: Setup and Installation

First, let's make sure we have all the required packages installed.

In [ ]:
# Install required packages (run this once)
# Uncomment the line below if packages are not installed
# !pip install openai python-dotenv pypdf

# Note: The 'openai' package is used for both OpenAI API and Ollama
# (Ollama has an OpenAI-compatible API)

In [ ]:
# Import libraries
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Imports successful!")

### Quick Setup Check

Make sure your `.env` file is configured. For Ollama (free):
```
LLM_PROVIDER=ollama
OLLAMA_MODEL=llama3.2
```

For OpenAI (requires API key):
```
LLM_PROVIDER=openai
OPENAI_API_KEY=sk-your-key-here
```

## Step 2: Understanding PDFs and Text Extraction

Before we can chat about a PDF, we need to extract its text content.

In [ ]:
import pypdf

# Let's extract text from a PDF
pdf_path = "../data/sample.pdf"  # Change this to your PDF

# Check if file exists
if Path(pdf_path).exists():
    with open(pdf_path, 'rb') as file:
        pdf_reader = pypdf.PdfReader(file)
        
        print(f"📄 PDF Info:")
        print(f"   Pages: {len(pdf_reader.pages)}")
        print(f"   Metadata: {pdf_reader.metadata}")
        
        # Extract first page text
        first_page = pdf_reader.pages[0]
        text = first_page.extract_text()
        
        print(f"\n📝 First 500 characters:")
        print(text[:500])
else:
    print("❌ PDF file not found!")
    print("💡 Add a PDF to the data/ folder and update the path above")

## Step 3: Understanding LLMs (Large Language Models)

### What is an LLM?
- A neural network trained on massive amounts of text
- Predicts the next word/token based on previous context
- Examples: GPT-4, Claude, Llama, Gemini

### Key Concepts:
- **Tokens**: Words or word pieces (1 token ≈ 4 characters)
- **Context Window**: Max tokens the model can process at once
- **Temperature**: Controls randomness (0=deterministic, 1=creative)
- **System Prompt**: Instructions for how the AI should behave
- **User Prompt**: Your actual question or request

In [ ]:
from openai import OpenAI

# Choose your provider: 'ollama' (free, local) or 'openai' (paid, cloud)
provider = os.getenv("LLM_PROVIDER", "ollama")

if provider == "ollama":
    # Initialize Ollama client (no API key needed!)
    client = OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama"  # Dummy key (Ollama doesn't need real keys)
    )
    model = os.getenv("OLLAMA_MODEL", "llama3.2")
    print(f"🦙 Using Ollama with {model}")
    print("💡 Make sure Ollama is running: ollama serve")
else:
    # Initialize OpenAI client
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    model = "gpt-4o-mini"
    print(f"🤖 Using OpenAI with {model}")

# Simple example: asking the LLM a question
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful teacher."},
        {"role": "user", "content": "Explain what an AI agent is in one simple sentence."}
    ],
    temperature=0.7
)

answer = response.choices[0].message.content
print(f"\n🤖 AI Response:\n{answer}")

# Show token usage if available (OpenAI provides this, Ollama may not)
if hasattr(response, 'usage') and response.usage:
    print(f"\n📊 Tokens used: {response.usage.total_tokens}")

## Step 4: Context Augmented Generation (CAG)

### The Magic of CAG:
1. Extract text from PDF
2. Send the text + question to the LLM
3. LLM answers based on the provided context

### CAG vs RAG:
- **CAG**: Send entire document as context (simple, works for smaller docs)
- **RAG**: Search for relevant chunks, only send those (better for large docs)

For learning, CAG is perfect!

In [ ]:
# Let's extract full PDF text
def extract_full_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = pypdf.PdfReader(file)
        text_parts = []
        
        for page in pdf_reader.pages:
            text_parts.append(page.extract_text())
        
        return "\n\n".join(text_parts)

# Try it (if you have a PDF)
if Path(pdf_path).exists():
    full_text = extract_full_pdf(pdf_path)
    
    print(f"📊 Extraction Stats:")
    print(f"   Characters: {len(full_text):,}")
    print(f"   Words: {len(full_text.split()):,}")
    print(f"   Est. Tokens: {len(full_text) // 4:,}")
    
    # Store for next step
    pdf_content = full_text
else:
    print("Add a PDF to test this!")

## Step 5: Building the Chatbot!

Now let's put it all together and create our PDF chatbot.

In [ ]:
def ask_pdf_question(question, pdf_content):
    """
    Ask a question about the PDF content.
    This is CAG in action!
    """
    # Get provider settings
    provider = os.getenv("LLM_PROVIDER", "ollama")
    
    if provider == "ollama":
        client = OpenAI(
            base_url="http://localhost:11434/v1",
            api_key="ollama"
        )
        model = os.getenv("OLLAMA_MODEL", "llama3.2")
    else:
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        model = "gpt-4o-mini"
    
    # Build the messages
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant that answers questions based on "
                "the provided document. Use only information from the document. "
                "If the answer isn't in the document, say so clearly."
            )
        },
        {
            "role": "system",
            "content": f"Document content:\n\n{pdf_content}"
        },
        {
            "role": "user",
            "content": question
        }
    ]
    
    # Get response
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=1000
    )
    
    answer = response.choices[0].message.content
    
    # Get tokens if available
    tokens = None
    if hasattr(response, 'usage') and response.usage:
        tokens = response.usage.total_tokens
    
    return answer, tokens

In [ ]:
# Test the chatbot!
if 'pdf_content' in locals():
    question = "What is this document about? Give me a brief summary."
    
    print(f"❓ Question: {question}\n")
    answer, tokens = ask_pdf_question(question, pdf_content)
    print(f"🤖 Answer: {answer}")
    
    if tokens:
        print(f"\n📊 Tokens used: {tokens}")
else:
    print("Add a PDF first to test the chatbot!")

## Step 6: Try Your Own Questions!

Now you can ask any question about your PDF.

In [ ]:
# Interactive question answering
if 'pdf_content' in locals():
    # Try different questions:
    questions = [
        "What are the main topics covered?",
        "List any key concepts or terms",
        "What's the most important takeaway?"
    ]
    
    for q in questions:
        print(f"\n{'='*60}")
        print(f"❓ {q}")
        print('='*60)
        answer, _ = ask_pdf_question(q, pdf_content)
        print(f"🤖 {answer}")
else:
    print("Add a PDF to the data/ folder to start chatting!")

## 🎓 What You've Learned!

Congratulations! You've now built a working AI chatbot. Here's what you learned:

1. **PDF Processing**: How to extract text from documents
2. **LLM Interaction**: How to send prompts and get responses (locally or via API)
3. **CAG**: How to use context to make AI more accurate
4. **Prompt Engineering**: Crafting system and user prompts
5. **Free AI**: Using Ollama to run models locally without costs

## 🚀 Next Steps

Ready to level up? Here are ideas:

1. **Try Different Models**: Use `ollama pull qwen2.5:7b` for better quality
2. **Add RAG**: Learn vector databases (ChromaDB) for large documents
3. **Memory**: Make the chatbot remember conversation history
4. **Multi-PDF**: Handle multiple documents at once
5. **Web Interface**: Build a UI with Streamlit or Gradio
6. **Advanced Agents**: Add tools, function calling, chains

## 📚 Resources

- [Ollama](https://ollama.ai/) - Free local models (what we used!)
- [Ollama Model Library](https://ollama.ai/library) - Browse available models
- [OpenAI API Documentation](https://platform.openai.com/docs) - If using OpenAI
- [LangChain](https://python.langchain.com/) - Framework for LLM apps
- [ChromaDB](https://www.trychroma.com/) - Vector database for RAG

## 💡 Model Recommendations

### For PDF Q&A:
- **llama3.2** (2GB) - Great balance, 128K context window ⭐
- **qwen2.5:7b** (4.7GB) - Better for technical docs
- **phi3:medium** (7.9GB) - Best quality (needs 16+ GB RAM)

Try different models:
```bash
ollama pull qwen2.5:7b
```

Then change `.env`: `OLLAMA_MODEL=qwen2.5:7b`

## 💡 Exercises

Try these challenges:

1. **Model Comparison**: Try llama3.2, qwen2.5:7b, and phi3 on the same question - compare speed and quality
2. **Temperature Experiments**: Modify the temperature parameter (0.0-1.0) - how does it change responses?
3. **System Prompts**: Try different system prompts - make it formal, casual, expert-level, or funny
4. **Context Window**: Count tokens in your PDF - will it fit in the 128K context window?
5. **Error Handling**: Add handling for when questions aren't answerable from the PDF
6. **Conversation History**: Store previous Q&A pairs and include them in context
7. **Multi-Model**: Build a function that asks multiple models and compares their answers